In [3]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_pinecone import PineconeVectorStore
from langchain_community.vectorstores import Pinecone
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import DirectoryLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter


from langchain_community.llms import CTransformers
import os
import pinecone
from pinecone import Pinecone
from dotenv import load_dotenv

In [5]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [6]:
load_dotenv()
key= os.getenv("PINECONE_API_KEY")

In [7]:
PINECONE_API_KEY = key
PINECONE_API_ENV = 'gcp-starter'

In [8]:
#extract data func
def load_pdf_data(data):
    loader = DirectoryLoader(data,
                    glob='*.pdf',
                    loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents

In [9]:
#path = '../'
extracted_data = load_pdf_data("../data")

In [10]:
#create chunks of text
def text_chunk_splitter(extracted_data):
    text_split = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunk = text_split.split_documents(extracted_data)

    return text_chunk

In [11]:
text_chunks = text_chunk_splitter(extracted_data)
print("chunk length:  ", len(text_chunks))

chunk length:   10484


In [12]:
#download embedding model
def download_embedding_model():
    embedding= HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    return embedding


In [13]:
embedding = download_embedding_model()

In [16]:
query_res = embedding.embed_query("Hi today is March 12 Tuesday")
print("Length: ", len(query_res))

Length:  384


In [17]:
from langchain_pinecone import PineconeVectorStore

In [18]:
pinecone.Pinecone(
   api_key=os.getenv("PINECONE_API_KEY"),  
   environment=os.getenv("PINECONE_ENV"),  
)
index_name = "medical-chatbot-vector-index"

In [ ]:
# Run this line when loading the embedding to the vector store first time
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embedding, index_name=index_name)

In [19]:
docsearch = PineconeVectorStore.from_existing_index(index_name, embedding)
query = "what is allergy?"
retiriever = docsearch.as_retriever(search_kwargs=dict(k=3))
docs = retiriever.get_relevant_documents(query)

print("Results", docs)
#docs[0].page_content
#[doc.page_content for doc in docs]

Results [Document(page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 2591Physical allergy'), Document(page_content='KEY TERMS\nAllergen —Any substance that irritates only those\nwho are sensitive (allergic) to it.\nAsthma —Wheezing (labored breathing) due to\nallergies or irritation of the lungs.\nDecongestant —Medicines that shrink blood ves-\nsels and consequently mucus membranes. Pseu-doephedrine, phenylephrine, and phenylpropano-lamine are the most common.\nSinus —Air-filled cavities surrounding the eyes and'), Document(page_content='most serious human immunodeficiency disorder(s). It isa group of congenital disorders in which both thehumoral part of the patient’s immune system and theKEY TERMS\nAllergy —Altered body reaction, usually hypersen-\nsitivity, as a response to exposure to a specific sub-stance.\nAntibody —Any of a large number of proteins that\nare produced after stimulation by an antigen andact specifically against the antigen in an immuneresponse.\nAntihistamine —A drug th

In [22]:
prompt_template = """ 
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.


Context : {context}
Question: {question}

Only return helpful answer below and nothing else.
Helpful answer:
"""

In [23]:
context = "\n\n".join([doc.page_content for doc in docs])
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
#PROMPT = ChatPromptTemplate(prompt_template) #,input_variables=["context", "question"])

#prompt = PROMPT.format(context=context,question= query) 
chain_type_kwargs = {"prompt": PROMPT}

In [24]:
# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

In [25]:
from llama_cpp import Llama

## N.B
CMAKE_ARGS="-DLLAMA_OPENBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.2.56


In [26]:
llm = LlamaCpp(
    model_path= "../model/gguf/llama-2-7b-chat.Q3_K_M.gguf",
    n_gpu_layers= -1,
    n_batch= 512,
    max_tokens= 512,
    temperature=0.7,
    callback_manager=callback_manager,
    #verbose=True # Verbose is required to pass to the callback manager
)

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../model/gguf/llama-2-7b-chat.Q3_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32  

In [28]:
QA = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever = retiriever,
    #retriever = docsearch.as_retriever(search_kwargs={'k':3}),
    return_source_documents = True,
    chain_type_kwargs=chain_type_kwargs
)


In [35]:
import textwrap

In [40]:
import sys
while True:
    user_input = input(f"Input prompt: ")
    if user_input == 'exit':
        print("Exiting")
        sys.exit()
    if user_input == ' ':
        continue
    result = QA({'query': user_input})
    result = textwrap.fill(result["result"], width=50)
    #print(f"Answer: {wraped_result['wraped_result']}")
    print(f"Answer: {result}")

Llama.generate: prefix-match hit


Insulin resistance does not affect sleep directly. Insulin resistance is a metabolic disorder in which the body's cells become less responsive to insulin, leading to high blood sugar levels. While there may be indirect effects of insulin resistance on sleep, such as increased inflammation or altered hormone levels, there is no direct link between insulin resistance and sleep disorders.


llama_print_timings:        load time =   12722.60 ms
llama_print_timings:      sample time =      35.66 ms /    89 runs   (    0.40 ms per token,  2496.00 tokens per second)
llama_print_timings: prompt eval time =    9850.69 ms /   304 tokens (   32.40 ms per token,    30.86 tokens per second)
llama_print_timings:        eval time =   15769.92 ms /    88 runs   (  179.20 ms per token,     5.58 tokens per second)
llama_print_timings:       total time =   26473.65 ms /   392 tokens


Answer: Insulin resistance does not affect sleep directly.
Insulin resistance is a metabolic disorder in
which the body's cells become less responsive to
insulin, leading to high blood sugar levels. While
there may be indirect effects of insulin
resistance on sleep, such as increased
inflammation or altered hormone levels, there is
no direct link between insulin resistance and
sleep disorders.
Exiting


SystemExit: 

/Users/mtm007/Downloads/LLM/Medchatbot/RAG_MedChatBot/Rchat/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
# while True:
#     user_Input: input(f"Input prompt: ")
#     result = QA({"query": user_Input})
#     print("Response: ", result["result"])